## Heat Capacity Analysis
- takes ```.log``` files which contain total energy times at various time steps over the course of the 200ps MD simulation to compute heat capacity using this formula:

$$
C_v = \left( \frac{\partial E}{\partial T} \right)_{N,V,T} = \frac{\langle E^2 \rangle - \langle E \rangle^2}{k_B T^2}
$$

- should be pretty quick to run since this doesn't require running the entire MD simulation all over again. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# Parameters (adjust as needed)
T = 300
timestep = 0.25  # fs
nsteps = 200000
N = 64 
kB = 8.617333262e-5  # Boltzmann constant in eV/K
log_dir = './traj_1fs_files/'

eV_to_J = 1.602176634e-19         # J per eV [1]
kB_J = 1.380649e-23               # J/K [2]
N_A = 6.02214076e23               # mol^-1 [3]


In [2]:
def analyze_energy_file(log_file_path, functional, n_blocks=10):
    """
    Analyze energy data from a single log file with block-wise error estimation
    
    Parameters:
    log_file_path (str): Full path to the log file
    functional (str): Name of the functional extracted from filename
    n_blocks (int): Number of blocks for error estimation (default=10)
    
    Returns:
    dict: Analysis results including error estimate
    """
    # Create output paths based on functional name
    folder_path = os.path.dirname(log_file_path)
    output_dir = "/global/scratch/users/namdao2404/heat_capacity/energy_outputs/"
    os.makedirs(output_dir, exist_ok=True)
    output_txt_path = os.path.join(output_dir, f'log_cv_{functional}.txt')
    
    try:
        col_names = ['Time[ps]', 'Etot[eV]', 'Epot[eV]', 'Ekin[eV]', 'T[K]']
        data = pd.read_csv(
            log_file_path,
            sep=r'\s+',
            skiprows=2,
            names=col_names
        )
        
        tot_energy = data['Etot[eV]'].values
        temperatures = data['T[K]'].values

        half_point = len(tot_energy) // 2
        E_eq = tot_energy[half_point:]
        T_eq = np.mean(temperatures[half_point:])
        print(T_eq)
        T_eq = 300
        
        # Calculate heat capacity for entire system
        mean_E = np.mean(E_eq)
        mean_E2 = np.mean(E_eq**2)
        variance_total = mean_E2 - mean_E**2
        
        # Convert energies from eV to J
        E_eq_J = E_eq * eV_to_J
        variance_J2 = np.mean(E_eq_J**2) - np.mean(E_eq_J)**2
        cv_system_J_per_K = variance_J2 / (kB_J * T_eq**2)
        cv_molar = (cv_system_J_per_K / N) * N_A
        
        # Block-wise error estimation
        block_size = len(E_eq) // n_blocks
        block_cv_molar = []
        
        for i in range(n_blocks):
            block_data = E_eq[i*block_size:(i+1)*block_size]
            block_data_J = block_data * eV_to_J
            mean_block = np.mean(block_data_J)
            mean_block2 = np.mean(block_data_J**2)
            var_block = mean_block2 - mean_block**2
            cv_block = var_block / (kB_J * T_eq**2)
            block_cv_molar.append((cv_block / N) * N_A)
        
        # Calculate standard error of the mean
        cv_error = np.std(block_cv_molar) / np.sqrt(n_blocks)
        
        # Save results with error estimate
        with open(output_txt_path, 'w') as f:
            f.write(f"Functional: {functional}\n")
            f.write(f"Temperature: {T_eq} K\n")
            f.write(f"Particles (N): {N}\n")
            f.write(f"Mean total energy: {mean_E:.6f} eV\n")
            f.write(f"Variance of total energy: {variance_total:.6f} eV²\n")
            f.write(f"MOLAR heat capacity (Cv): {cv_molar:.6f} J/(mol·K)\n")
            f.write(f"Heat capacity error estimate: {cv_error:.6f} J/(mol·K)\n")
        
        return {
            'functional': functional,
            'mean_tot_energy': mean_E,
            'variance_total': variance_total,
            'cv_molar': cv_molar,
            'cv_error': cv_error,
            'output_txt': output_txt_path
        }
        
    except FileNotFoundError:
        print(f"Error: File not found: {log_file_path}")
        return None
    except Exception as e:
        print(f"Error processing {log_file_path}: {str(e)}")
        return None
